<a href="https://colab.research.google.com/github/sturu1/Mon-AM/blob/master/2020_08_03_%EC%98%A4%EC%A0%84%EC%88%98%EC%97%85_%EC%9D%B4%EB%AF%B8%EC%A7%80%EB%B6%84%ED%95%A0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 이미지 분할

https://chloevan.github.io/python/tensorflow2.0/ch9_5_image_segmentation/

https://medium.com/hyunjulie/1%ED%8E%B8-semantic-segmentation-%EC%B2%AB%EA%B1%B8%EC%9D%8C-4180367ec9cb

https://www.whydsp.org/318

https://divamgupta.com/image-segmentation/2019/06/06/deep-learning-semantic-segmentation-keras.html